In [29]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

from torch import nn
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

import itertools

В этом ноутбуке предлагается обучить модель классификации изображений датасета CIFAR10, который содержит 60к цветных картинок разрешения 32х32 принадлежащих 10 классам.

Загрузим датасет

In [30]:
def get_load_data(batch_size):
  transform_train = transforms.Compose([
      transforms.RandomCrop(32, padding=4),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  transform_test = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  # batch_size = 512

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
  testloader = torch.utils.data.DataLoader(testset, batch_size=len(testset), shuffle=False, num_workers=2)

  classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  return trainset, trainloader, testset, testloader, classes

In [31]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def plot_images():
  dataiter = iter(trainloader)
  images, labels = dataiter.next()

  imshow(torchvision.utils.make_grid(images[:4]))
  print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [32]:
# оригинальная нейронная сеть из задания
class Net_org(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size = 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.Tanh()(self.conv1(x)))
        x = self.pool(nn.Tanh()(self.conv2(x)))
        x = nn.Flatten()(x)
        x = nn.Tanh()(self.fc1(x))
        x = nn.Tanh()(self.fc2(x))
        x = nn.Softmax()(self.fc3(x))
        return x

In [33]:
# изменяю саму модель на AlexNet
class Net_AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256 * 2 * 2, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)
        self.drop = nn.Dropout()

    def forward(self, x):
        x = self.pool(nn.LeakyReLU(0.1)(self.conv1(x)))
        x = self.pool(nn.LeakyReLU(0.1)(self.conv2(x)))
        x = nn.LeakyReLU(0.1)(self.conv3(x))
        x = nn.LeakyReLU(0.1)(self.conv4(x))
        x = self.pool(nn.LeakyReLU(0.1)(self.conv5(x)))
        # x = nn.Flatten()(x)
        x = self.drop(x.view(x.size(0), 256 * 2 * 2))
        x = nn.LeakyReLU(0.1)(self.fc1(x))
        x = self.drop(x)
        x = nn.LeakyReLU(0.1)(self.fc2(x))
        x = self.fc3(x)
        return x

In [34]:
# создаем список из нескольких нейронных сетей (для выбора лучшей)
net_org = Net_org().cuda() # оригинальная сеть из задания
net_AlexNet = Net_AlexNet().cuda()

# создаем список сетей для прохода циклом по ним
nets = [net_org, net_AlexNet]

In [35]:
# ссылка на список официальных функций потерь
# https://pytorch.org/docs/stable/nn.html#loss-functions

# создам список потерь
crit_CrEn = nn.CrossEntropyLoss()
# crit_MSELoss=nn.MSELoss()
# crit_GaNLL = nn.GaussianNLLLoss()

# пока оставил одну функцию для написания логики
criterions = [crit_CrEn]#, crit_MSELoss]#, crit_GaNLL]

In [36]:
# подозреваю, что так нельзя делать (в этом блоке ниже), тк оптимизаторы получаются привязанными к одной модели
# поэтому сделал ниже функцию по активации оптимизаторов в зависимости от модели
net=nets[0]
opt_Adam = optim.Adam(net.parameters())
opt_SGD = optim.SGD(net.parameters(), lr=0.1)
opt_Momentum = optim.SGD(net.parameters(),lr=0.1, momentum=0.8)

# попробую другие оптимизаторы (их много...)
# optimizer = optim.Adadelta(net.parameters())
# optimizer = optim.ASGD(net.parameters())
# optimizer = optim.NAdam(net.parameters())
# optimizer = optim.RAdam(net.parameters())
 
optimizers = [opt_Adam, opt_SGD, opt_Momentum]

In [59]:
def get_optimizers(net):
  # пока пробую на трех оптимизаторах
  opt_Adam = optim.Adam(net.parameters())
  opt_SGD = optim.SGD(net.parameters(), lr=0.1)
  opt_Momentum = optim.SGD(net.parameters(),lr=0.1, momentum=0.8)

  # 
  optimizers = [opt_Adam, opt_SGD, opt_Momentum]
  optimizers_names = ['opt_Adam', 'opt_SGD', 'opt_Momentum']
  return optimizers, optimizers_names

In [37]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

Обучим модель

In [11]:
#@title
# оригинальный блок обучения
for epoch in range(1):  
  
  for X_batch, y_batch in tqdm(trainloader):
      X_batch, y_batch = X_batch.cuda(), y_batch.cuda()

      optimizer.zero_grad()

      predictions = net(X_batch)

      loss = criterion(predictions, y_batch)
      loss.backward()

      optimizer.step()


epoch: 1


100%|██████████| 98/98 [00:20<00:00,  4.84it/s]


In [62]:
# основной блок по обучению и расчета показателя
batch_size = 512
print('Устанавливаем batch_size=',batch_size)

# циклом проходим по всем комбинациям
for r in itertools.product(nets, criterions):
  torch.cuda.empty_cache() # чистим кэш (у меня вылезала ошибка памяти)

  # получаем модель и функцию потерь
  net = r[0]
  criterion = r[1]

  # пересоздаем оптимизаторы под выбранную модель
  optimizers, optimizers_names = get_optimizers(net)

  # теперь проходим по всем оптимизаторам
  for i in range(0,len(optimizers)):
    # print(optimizers_names[i])
    optimizer = optimizers[i]

    print(str(net).split('(')[0],'-',criterion,'-',str(optimizers_names[i]))

    # print('Теперь перезагружаем исходные данные')
    trainset, trainloader, testset, testloader, classes = get_load_data(batch_size)
    # print('Загрузка закончена')

    # print('Отрисуем несколько картинок, посмотрим, как выглядит наш датасет')
    # plot_images()

    # теперь обучаем модель
    # эпоху поставил одну просто для ускорения (потом буду изменять, скорее всего на 10)
    for epoch in range(1):  
      
      for X_batch, y_batch in tqdm(trainloader):
          X_batch, y_batch = X_batch.cuda(), y_batch.cuda()

          optimizer.zero_grad()

          predictions = net(X_batch)

          loss = criterion(predictions, y_batch)
          loss.backward()

          optimizer.step()

    # посчитаем аккуратность обучения
    for X_batch, y_batch in testloader:
      X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
      with torch.no_grad():
          predictions = net(X_batch)

      true_lab = y_batch.cpu().numpy()
      _, preds = torch.max(predictions, dim=1)
      # print(true_lab)
      # print(preds)
      print('accuracy_score: ', accuracy_score(true_lab, preds.detach().cpu().numpy()))


opt_Adam
Net_org - CrossEntropyLoss() - opt_Adam
Устанавливаем batch_size= 512
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 98/98 [00:20<00:00,  4.83it/s]


accuracy_score:  0.52
opt_SGD
Net_org - CrossEntropyLoss() - opt_SGD
Устанавливаем batch_size= 512
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 98/98 [00:20<00:00,  4.89it/s]


accuracy_score:  0.5231
opt_Momentum
Net_org - CrossEntropyLoss() - opt_Momentum
Устанавливаем batch_size= 512
Files already downloaded and verified
Files already downloaded and verified


...

accuracy_score:  0.1
opt_SGD
Net_AlexNet - CrossEntropyLoss() - opt_SGD
Устанавливаем batch_size= 512
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 98/98 [00:21<00:00,  4.61it/s]


accuracy_score:  0.1
opt_Momentum
Net_AlexNet - CrossEntropyLoss() - opt_Momentum
Устанавливаем batch_size= 512
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 98/98 [00:21<00:00,  4.64it/s]


accuracy_score:  0.1


Запись результатов от ручного перебора  
(осталось с предыдущей версии файла)

оригинал: accuracy_score:  0.4194  
увеличил с 2 -> 5 число эпох: accuracy_score:  0.5182  
optimizer = optim.SGD(net.parameters(), lr=0.1) :  accuracy_score:  0.5227  
увеличил с 5 -> 10 число эпох: accuracy_score:  0.5468

вернул оптимизатор Adam, уменьшил число эпох до 5 и поставил relu как функцию активации: accuracy_score:  0.4219

10 эпох, relu и SGD (lr=0.01): accuracy_score:  0.1001  
10 эпох, relu и SGD (lr=0.1): accuracy_score: 0.3696

10 эпох, relu6 и SGD (lr=0.1): accuracy_score: 0.3596

10 эпох, Sigmoid и SGD (lr=0.1):accuracy_score:  0.1

посчитал на изначальной модели: accuracy_score:  0.5512

посчитал на своей модели: accuracy_score:  0.1089

исправил свою модель accuracy_score:  0.6117  
увеличил эпохи до 10: accuracy_score:  0.6686  

Adam.
relu: accuracy_score:  0.7006  
Tanh: accuracy_score:  0.6861  
Sigmoid: accuracy_score:  0.1  
LeakyReLU: accuracy_score:  0.7032  
LeakyReLU(0.1) :accuracy_score:  0.7357

Теперь попробую другие оптимизаторы для LeakyReLU(0.1):  
Adadelta: accuracy_score:  0.6185  
ASGD: accuracy_score:  0.1356  
NAdam: accuracy_score:  0.3775  
RAdam:  accuracy_score:  0.6209  
Momentum: accuracy_score:  0.1

